In [1]:
import numpy as np
from random import randrange

In [4]:
class smo_algo:
    def __init__(self,train_data_features,labels,reg_strength,tolerance):
        self.X = train_data_features
        self.c_labels = labels
        self.C = reg_strength
        self.tol = tolerance
        self.num_lagrange,self.theta_hat_dim = np.shape(self.X)
        self.lagrange_muls = np.zeros(self.num_lagrange)
        self.errors = np.zeros(self.num_lagrange)
        self.epsilon = 10**(-3)
        self.theta0_hat = 0
        self.theta_hat = np.zeros(self.theta_hat_dim)
        
    def discriminant_score(self,i):
        return self.theta0_hat + float(np.dot(self.theta_hat.T,self.X[i]))
        
    def compute_error(self,i2):
        return self.discriminant_score(i2) - self.c2
    
    def get_non_bound_indexes(self):
        return np.where(np.logical_and(self.lagrange_muls > 0,self.lagrange_muls<=self.C))[0]
    
    def get_maximum_change_lagrange(self,non_bound_indices):
        i1 = -1
        if len(non_bound_indices) > 1:
            max = 0
            
            for j in non_bound_indices:
                E1 = self.errors[j] - self.c_labels[j]
                step = abs(E1 - self.E2)
                if step > max:
                    max = step
                    i1 = j
        return i1
    
    def take_cordinate_ascent_step(self,i1,i2):
        if i1 == i2:
            return False
        
        lambda1 = self.lagrange_muls[i1]
        c1 = self.c_labels[i1]
        X1 = self.X[i1]
        E1 = self.compute_error(i1)
        
        labels_prod = c1 * self.c2
        
        if c1 != self.c2:
            L = max(0,self.lambda2-lambda1)
            H = min(self.C,self.lambda2+lambda1)
        else:
            L = max(0,self.lambda2+lambda1-self.C)
            H = min(self.C,self.lambda2+lambda1)
            
        if L == H:
            return False
        
        k11 = np.dot(X1.T,X1)
        k12 = np.dot(X1,self.X[i2])
        k22 = np.dot(self.X[i2],self.X[i2])
        
        second_derivative = k11 + k22 - k12
        
        if double_derivative > 0:
            lambda2_final = self.lambda2 + self.c2*(E1 - self.E2)/double_derivative
            
            if lambda2_final < L:
                lambda2_final = L
            elif lambda2_final > H:
                lambda2_final = H
                
        if abs(lambda2_final - self.lambda2)< self.epsilon:
            return False
        
        lambda1_final = lambda1 + labels_prod * (self.lambda2 - lambda2_final)
        theta0_hat_final = self.compute_theta0_hat(E1,lambda1,lambda1_final,lambda2_final,k11,k12,k22,c1)
        delta_theta0_hat = theta0_hat_final - self.theta0_hat
        self.theta0_hat = theta0_hat_final
        
        self.theta_hat = self.theta_hat + c1*(lambda1_final - lambda1)*X1 + self.c2*(lambda2_final - self.lambda2)*self.X2
        
        delta_i1 = c1*(lambda1_final - lambda1)
        delta_i2 = c2*(lambda2_final - lambda2)
        
        for i in range(self.num_lagrange):
            if self.lagrange_muls[i] > 0 and self.lagrange_muls[i] <= self.C:
                self.errors[i] = self.errors[i] + delta_i1 * np.dot(X1,self.X[i]) + delta_i2 * np.dot(X2,self.X[i]) - delta_theta0_hat
            
        self.errors[i1] = 0
        self.errors[i2] = 0
        self.lagrange_muls[i1] = lambda1_final
        self.lagrange_muls[i2] = lambda2_final
        
        return True
    
    
    def compute_theta0_hat(self,E1,lambda1,lambda1_final,lambda2_final,k11,k12,k22,c1):
        
        theta0_hat1 = E1 + c1*(lambda1_final - lambda1)*k11 + self.c2*(lambda2_final - self.lambda2)*k12 + self.theta0_hat
        theta0_hat2 = self.E2 + c1*(lambda1_final - lambda1)*k12 + self.c2*(lambda2_final - self.lambda2)*k22 + self.theta0_hat
        
        if lambda1_final > 0 and lambda1_final < self.C:
            theta0_hat_final = theta0_hat1
        elif lambda2_final > 0 and lambda2_final < self.C:
            theta0_hat_final = theta0_hat2
        else:
            theta0_hat_final = (theta0_hat1 + theta0_hat2)/2.0
        return theta0_hat_final
        
            
        
    def check_example(self,i2):
        self.c2 = self.c_label[i2]
        self.lambda2 = self.lagrange_muls[i2]
        self.X2 = self.X[i2]
        self.E2 = self.compute_error(i2)
        
        #Calculating the value of ci2*[theta0_hat + theta_hat.T*X2]-1
        constraint_diff2 = self.E2 * self.c2
        
        #the condition inside not will be true if both constraint as well as lagrange nultiplier will be 
        #greater than zero hence violating KKT dual complemtarity condition because at a time, either
        #constraint has to be zero within some +- tol or lagrange multiplier has to be zero. 
        if not((contraint_diff2 < -self.tol and self.lambda2 <= self.C) or 
              (constraint_diff2 > self.tol and self.lambda2 > 0)):
            return 0
        
        non_bound_lagrange_indices = list(self.get_non_bound_indexes())
        i1 = self.get_maximum_change_lagrange(non_bound_lagrange_indices)
        
        if i1>=0 and self.take_cordinate_ascent_step(i1,i2) == True:
            return 1
        
        if len(non_bound_lagrange_indices) > 0:
            random_range = randrange(len(non_bound_lagrange_indices))
            for i1 in non_bound_lagrange_indices[random_range:] + non_bound_lagrange_indices[:random_range]:
                if self.take_cordinate_ascent_step(i1,i2) == True:
                    return 1
                
        random_range = randrange(self.num_lagrange)
        all_indices = list(range(self.num_lagrange))
        for i1 in all_indices[random_range:] + all_indices[:random_range]:
            if self.take_cordinate_ascent_step(i1,i2) == True:
                return 1
            
        return 0
            
    def smo_algo_main_loop(self):
        
        num_changed_lagrange = 0
        check_all = True
        
        while num_changed_lagrange > 0 or check_all == True:
            num_changed_lagrange = 0
            
            if check_all == True:
                for i in range(self.num_lagrange):
                    num_changed_lagrange = num_changed_lagrange + self.check_example(i)
            else:
                num_changed_lagrange = num_changed_lagrange + self.check_non_bound_lagrange()
                
            if check_all == True:
                check_all = False
            elif num_lagrange_changed == 0:
                check_all = True